In [ ]:
!pip install tensorflow numpy

#
import urllib.request, numpy as np, tensorflow as tf
from tensorflow.keras import layers, callbacks

#
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
urllib.request.urlretrieve(url, 'shakespeare.txt')
with open('shakespeare.txt', 'r', encoding='utf-8') as f :
    text = f.read().lower()

chars = sorted(list(set(text)))
print("Chars : \n", chars)
char_to_idx = {c:i for i,c in enumerate(chars)}
idx_to_char = {i:c for i,c in enumerate(chars)}
vocab_size = len(chars)


#
seq_len = 50
step = 3
X, y = [], []
for i in range(0, len(text)-seq_len, step):
    X.append([char_to_idx[c] for c in text[i:i+seq_len]])
    y.append(char_to_idx[text[i+seq_len]])
print("X: \n", X)
print("Y: \n", y)
X = np.array(X)
y = tf.keras.utils.to_categorical(y, vocab_size)
print("X(Transformed): \n", X)
print("Y(Transformed): \n", y)




#
# === CHANGED MODEL SECTION ===
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 128, input_length=seq_len),
    layers.SimpleRNN(128, return_sequences=True),   # Changed from LSTM to SimpleRNN
    layers.Dropout(0.2),
    layers.SimpleRNN(128),                         # Changed from LSTM to SimpleRNN
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early = callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, batch_size=128, epochs=3, callbacks=[early], verbose=1)

loss, acc = model.evaluate(X, y, verbose=0)
print(f"Final Model Accuracy: {acc:.4f}")
print(f"Final Model Loss: {loss:.4f}")

#
def generate(seed: str, length: int=200, temperature = 1.0):
    seed_seq = seed.lower()[-seq_len:].ljust(seq_len, '\x00')
    x = np.array([char_to_idx.get(c, 0) for c in seed_seq]).reshape(1,seq_len)
    result = seed

    for _ in range(length):
        probs = model.predict(x, verbose=0)[0]
        probs = np.log(probs + 1e-12) / temperature
        probs = np.exp(probs) / np.sum(np.exp(probs))
        idx = np.random.choice(vocab_size, p=probs)
        result += idx_to_char[idx]
        x = np.roll(x, -1)
        x[0, -1] = idx
    return result
print("\n=== Sample generations ===")
for seed in ["to be or not to be", "the king ", "romeo, romeo, "]:
    print(f"\nSeed : {seed}")
    print("Gen  :", generate(seed, length=120, temperature=0.9))



#
model.build(input_shape=(None, seq_len))
model.summary()